In [1]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import utils
import numpy as np
import os
from collections import defaultdict
import pickle
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor()
    ]
)

In [4]:
last_activations = []
def get_last_layer_activation(module, input, output):
    last_activations.append(output)

## Resnet

### Edit Model for Last Layer Hook

In [10]:
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)

In [11]:
resnet18.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
resnet18.layer4[1]

BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [13]:
resnet18.layer4[1].register_forward_hook(get_last_layer_activation)

### Run Inference

In [50]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)
similaritiesdict = defaultdict(list)
last_activations = []

In [51]:
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if not 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['reversal']-------------------


['reversal']: 100%|██████████| 4094/4094 [01:31<00:00, 44.60it/s]


-------------Running ['mirror']-------------------


['mirror']: 100%|██████████| 4094/4094 [01:48<00:00, 37.71it/s]


-------------Running ['randomphasic']-------------------


...

['reversal', 'randomphasic']: 100%|██████████| 4094/4094 [01:29<00:00, 45.94it/s]


-------------Running ['randomdelete', 'reversal', 'randomphasic']-------------------


['randomdelete', 'reversal', 'randomphasic']: 100%|██████████| 4094/4094 [01:38<00:00, 41.70it/s]


-------------Running ['randomdelete', 'reversal', 'mirror']-------------------


['randomdelete', 'reversal', 'mirror']: 100%|██████████| 4094/4094 [01:38<00:00, 41.63it/s]


In [ ]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_12
with open(f'similarities/exhaustive/resnet/filled.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [15]:
pkl11path = 'mappings/exhaustive_mapping_11.pkl'
pkl11 = utils.load_dict(pkl11path)
similaritiesdict = defaultdict(list)
last_activations = []

In [16]:
folpath = 'exhaustive_generated_set\\11'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl11[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------


['randominsertion', 'randomphasic']: 100%|██████████| 2046/2046 [00:38<00:00, 52.91it/s]


-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------


['randominsertion', 'reversal', 'randomphasic']: 100%|██████████| 2046/2046 [00:39<00:00, 52.17it/s]


-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


['randominsertion', 'reversal', 'mirror']: 100%|██████████| 2046/2046 [00:40<00:00, 50.62it/s]


In [ ]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_11
with open(f'similarities/exhaustive/resnet/unfilled11.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [19]:
pkl10path = 'mappings/exhaustive_mapping_10.pkl'
pkl10 = utils.load_dict(pkl10path)
similaritiesdict = defaultdict(list)
last_activations = []

In [20]:
folpath = 'exhaustive_generated_set\\10'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl10[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------


['randominsertion', 'randomphasic']: 100%|██████████| 1022/1022 [00:29<00:00, 34.94it/s]


-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------


['randominsertion', 'reversal', 'randomphasic']: 100%|██████████| 1022/1022 [00:25<00:00, 40.35it/s]


-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


['randominsertion', 'reversal', 'mirror']: 100%|██████████| 1022/1022 [00:25<00:00, 39.39it/s]


In [22]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_10
with open(f'similarities/exhaustive/resnet/unfilled10.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

## VGG

### Hook Final Activation Layer

In [4]:
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).to(device)

In [5]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [9]:
vgg16.classifier[6].register_forward_hook(get_last_layer_activation)

### Run Inference

In [27]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if not 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                vgg16(imgin)
                vgg16(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['reversal']-------------------


['reversal']: 100%|██████████| 4094/4094 [01:48<00:00, 37.59it/s]


-------------Running ['mirror']-------------------


['mirror']: 100%|██████████| 4094/4094 [02:19<00:00, 29.39it/s]


-------------Running ['randomphasic']-------------------


...

['reversal', 'randomphasic']: 100%|██████████| 4094/4094 [02:28<00:00, 27.61it/s]


-------------Running ['randomdelete', 'reversal', 'randomphasic']-------------------


['randomdelete', 'reversal', 'randomphasic']: 100%|██████████| 4094/4094 [02:37<00:00, 25.93it/s]  


-------------Running ['randomdelete', 'reversal', 'mirror']-------------------


['randomdelete', 'reversal', 'mirror']: 100%|██████████| 4094/4094 [02:34<00:00, 26.49it/s]


In [28]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_12
with open(f'similarities/exhaustive/vgg/filled.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [10]:
pkl11path = 'mappings/exhaustive_mapping_11.pkl'
pkl11 = utils.load_dict(pkl11path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\11'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl11[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                vgg16(imgin)
                vgg16(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------


['randominsertion', 'randomphasic']: 100%|██████████| 2046/2046 [00:51<00:00, 39.95it/s]


-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------


['randominsertion', 'reversal', 'randomphasic']: 100%|██████████| 2046/2046 [00:51<00:00, 39.49it/s]


-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


['randominsertion', 'reversal', 'mirror']: 100%|██████████| 2046/2046 [00:51<00:00, 40.09it/s]


In [11]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_11
with open(f'similarities/exhaustive/vgg/unfilled11.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [12]:
pkl10path = 'mappings/exhaustive_mapping_10.pkl'
pkl10 = utils.load_dict(pkl10path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\10'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl10[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                vgg16(imgin)
                vgg16(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------


['randominsertion', 'randomphasic']: 100%|██████████| 1022/1022 [00:30<00:00, 33.53it/s]


-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------


['randominsertion', 'reversal', 'randomphasic']: 100%|██████████| 1022/1022 [00:24<00:00, 41.18it/s]


-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


['randominsertion', 'reversal', 'mirror']: 100%|██████████| 1022/1022 [00:25<00:00, 39.68it/s]


In [13]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_10
with open(f'similarities/exhaustive/vgg/unfilled10.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

## DenseNet

### Hook Final Activation Layer

In [5]:
densenet121 = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1).to(device)

In [6]:
densenet121.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
densenet121.classifier

Linear(in_features=1024, out_features=1000, bias=True)

In [8]:
densenet121.classifier.register_forward_hook(get_last_layer_activation)

#### Run Inference

In [9]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if not 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
        # for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                densenet121(imgin)
                densenet121(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['reversal']-------------------


['reversal']: 100%|██████████| 4094/4094 [04:26<00:00, 15.35it/s]


-------------Running ['mirror']-------------------


['mirror']: 100%|██████████| 4094/4094 [04:36<00:00, 14.81it/s]


-------------Running ['randomphasic']-------------------


...

['reversal', 'randomphasic']: 100%|██████████| 4094/4094 [05:00<00:00, 13.61it/s]  


-------------Running ['randomdelete', 'reversal', 'randomphasic']-------------------


['randomdelete', 'reversal', 'randomphasic']: 100%|██████████| 4094/4094 [04:55<00:00, 13.88it/s]


-------------Running ['randomdelete', 'reversal', 'mirror']-------------------


['randomdelete', 'reversal', 'mirror']: 100%|██████████| 4094/4094 [04:45<00:00, 14.34it/s]


In [10]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_12
with open(f'similarities/exhaustive/densenet/filled.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [11]:
pkl11path = 'mappings/exhaustive_mapping_11.pkl'
pkl11 = utils.load_dict(pkl11path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\11'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl11[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                densenet121(imgin)
                densenet121(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------


['randominsertion', 'randomphasic']: 100%|██████████| 2046/2046 [02:01<00:00, 16.79it/s]


-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------


['randominsertion', 'reversal', 'randomphasic']: 100%|██████████| 2046/2046 [03:12<00:00, 10.65it/s] 


-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


['randominsertion', 'reversal', 'mirror']: 100%|██████████| 2046/2046 [02:13<00:00, 15.28it/s]


In [12]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_11
with open(f'similarities/exhaustive/densenet/unfilled11.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [13]:
pkl10path = 'mappings/exhaustive_mapping_10.pkl'
pkl10 = utils.load_dict(pkl10path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\10'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl10[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                densenet121(imgin)
                densenet121(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------


['randominsertion', 'randomphasic']: 100%|██████████| 1022/1022 [01:12<00:00, 14.08it/s]


-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------


['randominsertion', 'reversal', 'randomphasic']: 100%|██████████| 1022/1022 [01:20<00:00, 12.69it/s]


-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


['randominsertion', 'reversal', 'mirror']: 100%|██████████| 1022/1022 [01:12<00:00, 14.14it/s]


In [14]:
for _, v in similaritiesdict.items():
    assert len(v) == utils.EXPECTED_LEN_10
with open(f'similarities/exhaustive/densenet/unfilled10.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

## Inception

### Hook Final Activation Layer

In [15]:
inception_v3 = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1).to(device)

In [16]:
inception_v3.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [17]:
inception_v3.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [18]:
inception_v3.fc.register_forward_hook(get_last_layer_activation)

### Run Inference

In [19]:
transform_inception = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((299, 299)),
        torchvision.transforms.ToTensor()
    ]
)

In [20]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if not 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in tqdm(os.listdir(folpath), desc=f'{transformation}'):
        # for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath).convert('RGB')
            imgin = transform_inception(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = Image.fromarray(utils.array2img(imglist, num_circles=len(imglist), radius=0.75)).convert('RGB')
            imgout = transform_inception(imgout).unsqueeze(0).to(device)
            # print(imgin.shape)
            with torch.no_grad():
                inception_v3(imgin)
                inception_v3(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['reversal']-------------------


['reversal']:   1%|          | 25/4094 [00:02<07:11,  9.42it/s]


KeyboardInterrupt: 

## AlexNet

In [ ]:
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1).to(device)

In [ ]:
alexnet

In [ ]:
alexnet.classifier

In [ ]:
alexnet.classifier[6].register_forward_hook(get_last_layer_activation)